In [3]:
import numpy as np
import pandas as pd
from patsy import dmatrices
import warnings
warnings.simplefilter("ignore")
from estimagic.optimization.utilities import sdcorr_params_to_matrix
from estimagic.optimization.utilities import sds_and_corr_to_cov
from estimagic.optimization.utilities import robust_cholesky
from estimagic.optimization.optimize import maximize

## Prepare User Input

In [4]:
df = pd.read_stata('sysdsn1.dta', convert_categoricals=False)
data = pd.concat([df, pd.get_dummies(df['site'], prefix='site')], axis=1)
formula = "insure ~ age + male + nonwhite + site_2 + site_3"
data['insure'].replace({1: 2, 2: 0, 3: 1}, inplace=True)

## Process User Input

In [5]:
def multinomial_processing(formula, data):
    """Construct the inputs for the multinomial functions.
    
    Args:
        formula (str): A patsy formula
        data (pd.DataFrame): The dataset
        
    Returns:
        y (np.array): 1d numpy array of shape (n_obs) with the observed choices
        x (np.array): 2d numpy array of shape (n_obs, n_var) with independent variables
        params (pd.DataFrame): Naive starting parameters.
        
    """
    # process data
    relevant = pd.concat(dmatrices(formula, data, return_type='dataframe'), axis=1).dropna()
    y, x = dmatrices(formula, relevant, return_type='dataframe')
    y = (y - y.min()).to_numpy(dtype=np.int64).flatten()
    x_variables = x.columns
    x = x.to_numpy(dtype=np.float64)
    
    # create params index
    n_choices = len(np.unique(y))
    index_tuples = []
    for c in range(n_choices - 1):
        index_tuples += [('beta', f'choice_{c}', x) for x in x_variables]
    index_tuples += [('shocks', 'sd', str(c)) for c in range(n_choices - 1)]
    index_tuples += [('shocks', 'corr', f'{i1 + 1}_{i2}') for i1, i2 in zip(*np.tril_indices(n_choices - 2))]
    
    # create the basic params df
    params = pd.DataFrame(
        index=pd.MultiIndex.from_tuples(
            index_tuples, names=['category', 'subcategory', 'name']))
    np.random.seed(5471)
    params['value'] = np.random.uniform(low=-0.02, high=0.02, size=len(params))
    params.loc[('shocks', 'sd'), 'value'] = 1
    
    # Add optional group column
    params['group'] = params.index.get_level_values('subcategory')
    params.loc['shocks', 'group'] = 'shocks'
    
    # Add optional bounds
    params['lower'] = - np.inf
    params['upper'] = np.inf
    
    return params, y, x

In [6]:
multinomial_processing(formula, data)[0]

value     group  lower  upper
category subcategory name                                       
beta     choice_0    Intercept  0.019095  choice_0   -inf    inf
                     age        0.009060  choice_0   -inf    inf
                     male      -0.018698  choice_0   -inf    inf
                     nonwhite   0.018672  choice_0   -inf    inf
                     site_2    -0.006744  choice_0   -inf    inf
                     site_3     0.011364  choice_0   -inf    inf
         choice_1    Intercept -0.010701  choice_1   -inf    inf
                     age        0.012593  choice_1   -inf    inf
                     male      -0.006271  choice_1   -inf    inf
                     nonwhite  -0.010662  choice_1   -inf    inf
                     site_2     0.005172  choice_1   -inf    inf
                     site_3    -0.013969  choice_1   -inf    inf
shocks   sd          0          1.000000    shocks   -inf    inf
                     1          1.000000    shocks   -inf    inf
         corr        1_0        0.006285    shocks   -inf    inf

## Likelihood Function

In [7]:
def mprobit_loglike(params, y, x):
    """Construct Log-likelihood contribution per individual of a probit model.
    
    Args:
        params (pd.DataFrame): The parameters of the model. 
        y (np.array): 1d numpy array of shape (n_obs) with the observed choices
        x (np.array): 2d numpy array of shape (n_obs, n_var) with independent variables
        
    Returns:
        loglikeobs (np.array): 1d numpy array of shape (n_obs) with likelihood 
            contribution per individual.
            
    """
    # extract dimensions
    n_var = x.shape[1]
    n_choices = len(np.unique(y))
    
    # parse the parameter vector
    sdcorr_params = params.loc['shocks', 'value']
    cov = sdcorr_params_to_matrix(sdcorr_params)
    betas = params.loc['beta', 'value'].unstack().T
    betas = np.column_stack([betas, np.zeros(n_var)])
    
    # actual calculations
    u_prime = x.dot(betas)
    choice_prob_obs = mc_integration(u_prime, cov, y)
    choice_prob_obs[choice_prob_obs<=1e-250] = 1e-250
    loglikeobs = np.log(choice_prob_obs)
    loglike = loglikeobs.sum()

    return loglike


def mc_integration(u_prime, cov, y, n_draws=None):
    """Calculate probit choice probabilities with Monte-Carlo Integration.
    
    Args:
        u_prime (np.array): 2d array of shape (n_obs, n_choices) with the
            deterministic part of utilities
        cov (np.array): 2d array of shape (n_choices - 1, n_choices - 1) with 
            the cov matrix
        y (np.array): 1d array of shape (n_obs) with the observed choices
        n_draws (int): Number of draws for Monte-Carlo integration.
        
    Returns:
        choice_prob_obs (np.array): 1d array of shape (n_obs) with the choice 
        probabilities for the chosen alternative for each individual.
        
    """
    # extract dimensions
    n_obs = np.shape(u_prime)[0]
    n_choices = np.shape(u_prime)[1]
    n_draws = n_choices * 2000 if n_draws is None else n_draws
    

    # generate the shocks
    np.random.seed(1995)
    base_error = np.random.normal(size=(n_obs*n_draws, (n_choices - 1)))
    chol = robust_cholesky(cov)
    errors = chol.dot(base_error.T)
    errors = errors.T.reshape(n_obs, n_draws, (n_choices - 1))
    extra_column_errors = np.zeros((n_obs, n_draws, 1))
    errors = np.append(errors, extra_column_errors, axis=2)

    # calculate utilities
    u = u_prime.reshape(n_obs, 1, n_choices) + errors
    
    # count simulated choices
    index_choices = np.argmax(u, axis=2)
    choices = np.zeros((n_obs, n_draws, n_choices))
    for i in range(n_obs):
        for j in range(len(index_choices[1])):
            choices[i, j, int(index_choices[i, j])] = 1
    choice_probs = np.average(choices, axis=1)
    choice_prob_obs = choice_probs[range(len(y)), y]
    
    return choice_prob_obs

## Notes

Naive Monte-Carlo integration is a terrible choice for calculating choice probabilities in a multinomial probit model. I chose it here, because it's so intuitive and short. If you are interested in better implementations of a multinomial probit model check out the [discrete choice](https://github.com/OpenSourceEconomics/discrete_choice) repo on OpenSourceEconomics. It's still in development but already usable. 

## Putting it Together

In [8]:
def mprobit(formula, data, algorithm='nlopt_bobyqa', dashboard=True):
    """Estimate ordered logit model described by *formula* on *data*.

    Args:
        forumla (str): A valid patsy formula
        data (DataFrame)
        algorithm (str): The optimization algorithm
        dashboard (bool)

    Returns:
        params (DataFrame): Estimated parameters

    """
    params, y, x = multinomial_processing(formula, data)
    
    constraints = [
        {'loc': 'shocks', 'type': 'sdcorr', 'bounds_distance': 1e-5},
        {'loc': ('shocks', 'sd', '0'), 'type': 'fixed', 'value': 1.0}
        ]
        
    info, estimated_params = maximize(
        criterion=mprobit_loglike, 
        params=params, 
        algorithm=algorithm, 
        criterion_kwargs={'y': y, 'x': x},
        constraints=constraints, 
        dashboard=dashboard,
    )
    
    return estimated_params

## Use the mprobit Function

In [ ]:
params = mprobit(formula, data)

Bokeh app running at: http://localhost:54447/
